In [1]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.insert(0, top)
# sys.path

In [12]:
import aiohttp
from pyquery import PyQuery as pq
import pandas as pd

In [8]:

# 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}

url = 'https://www.cac.edu.tw/CacLink/apply110/110apply_pgSieve_22sd8rga/html_sieve_110swk5m/ColPost/collegeList.htm'

#
connector = aiohttp.TCPConnector(ssl=True)
TO = aiohttp.ClientTimeout(total=10)
# 
async with aiohttp.ClientSession(connector=connector, timeout=TO) as session:
    async with session.get(url, headers=headers) as r:
        status = r.status
        rtext = await r.text(encoding='utf8')
        doc = pq(rtext, parser='html')

In [13]:
schools = doc.find('.htmldw1C')
tmp = []
for sch in schools:
    schname = pq(sch).text().strip()
    no, name = schname.split(")")
    tmp.append({'schno':no.strip('('),'schname':name})
df_sch = pd.DataFrame(tmp)
df_sch

,schno,schname
0,001,國立臺灣大學
1,003,國立中興大學
2,002,國立臺灣師範大學
3,004,國立成功大學
4,006,國立政治大學
...,...,...
64,151,國立聯合大學
65,150,國立宜蘭大學
66,152,馬偕醫學院
67,154,臺北基督學院


In [33]:
# 招生
type1 = doc.find('.htmldw1E a')
tmp = []
for a in type1:
    href = pq(a).attr('href')
    schno = href.strip('common/').strip('.html')
    tmp.append({'schno':schno,'type':'common'})
# 
df_type1 = pd.DataFrame(tmp).sort_values('schno')
df_type1

,schno,type
1,001,common
2,002,common
0,003,common
4,004,common
5,005,common
...,...,...
65,150,common
63,151,common
67,152,common
68,153,common


In [34]:
# 外加
type2 = doc.find('.htmldw1D a')
tmp = []
for a in type2:
    href = pq(a).attr('href')
    schno = href.strip('extra/').strip('.html')
    tmp.append({'schno':schno,'type':'extra'})
# 
df_type2 = pd.DataFrame(tmp).sort_values('schno')
df_type2

,schno,type
1,001,extra
2,002,extra
0,003,extra
4,004,extra
5,005,extra
...,...,...
63,134,extra
64,150,extra
62,151,extra
65,152,extra


In [49]:
tmp = 'https://www.cac.edu.tw/CacLink/apply110/110apply_pgSieve_22sd8rga/html_sieve_110swk5m/ColPost/'
# 
df_sch_type = pd.concat([df_sch.merge(df_type1),df_sch.merge(df_type2)]).sort_values(by=['schno','type'])
df_sch_type['url'] = tmp + df_sch_type['type'] + '/'+ df_sch_type['schno']+'.html'
df_sch_type

,schno,schname,type,url
0,001,國立臺灣大學,common,https://www.cac.edu.tw/CacLink/apply110/110app...
0,001,國立臺灣大學,extra,https://www.cac.edu.tw/CacLink/apply110/110app...
2,002,國立臺灣師範大學,common,https://www.cac.edu.tw/CacLink/apply110/110app...
2,002,國立臺灣師範大學,extra,https://www.cac.edu.tw/CacLink/apply110/110app...
1,003,國立中興大學,common,https://www.cac.edu.tw/CacLink/apply110/110app...
...,...,...,...,...
66,152,馬偕醫學院,common,https://www.cac.edu.tw/CacLink/apply110/110app...
65,152,馬偕醫學院,extra,https://www.cac.edu.tw/CacLink/apply110/110app...
68,153,國立金門大學,common,https://www.cac.edu.tw/CacLink/apply110/110app...
66,153,國立金門大學,extra,https://www.cac.edu.tw/CacLink/apply110/110app...


In [47]:
df_sch_type[:2].url.values

array(['https://www.cac.edu.tw/CacLink/apply110/110apply_pgSieve_22sd8rga/html_sieve_110swk5m/ColPost/common/001.html',
       'https://www.cac.edu.tw/CacLink/apply110/110apply_pgSieve_22sd8rga/html_sieve_110swk5m/ColPost/extra/001.html'],
      dtype=object)